### 1.Contextualized Topic Models, CTM 설치

In [1]:
%%capture
!pip install contextualized-topic-models==2.2.0 -qqq

In [2]:
%%capture
!pip install pyldavis -qqq

In [3]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 109 (delta 7), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (109/109), 1.27 MiB | 15.29 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4 MB 426 kB/s 
     |████████████████████████████████| 448 kB 42.9 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-03-28 04:41:36--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22e9:9f55, 2406:da00:ff00::22c0:3470, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Loc

### 2.노트북 재시작

In [ ]:
# import
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
# from konlpy.tag import Mecab
from konlpy.tag import Okt
from tqdm import tqdm
import pandas as pd

### 3.데이터 불러오기

In [ ]:
def listToString(s):  
    str1 = ""  
    for ele in s:  
        str1 += " " + ele.strip()  
    return str1

In [ ]:
df = pd.read_excel('./22Y_11W_naver_rawdata.xlsx',0)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  186 non-null    int64         
 1   DATE        186 non-null    datetime64[ns]
 2   WEEK        186 non-null    object        
 3   GUBUN       186 non-null    object        
 4   GUBUN1      0 non-null      float64       
 5   GUBUN2      0 non-null      float64       
 6   NAME        186 non-null    object        
 7   URL         186 non-null    object        
 8   TITLE       186 non-null    object        
 9   TEXT        186 non-null    object        
 10  PRODUCT     186 non-null    object        
 11  IMG_OBJECT  0 non-null      float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(7)
memory usage: 17.6+ KB


In [ ]:
# Text Cleansing
doc = listToString(df.TEXT)

In [ ]:
documents =list(df.TEXT)

In [ ]:
# documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()]

In [ ]:
preprocessed_documents = []

for line in tqdm(documents):
  # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
  if line and not line.replace(' ', '').isdecimal():
    preprocessed_documents.append(line)

100%|██████████| 186/186 [00:00<00:00, 13784.86it/s]


In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [ ]:
custom_tokenizer = CustomTokenizer(Okt())
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
train_bow_embeddings = vectorizer.fit_transform(preprocessed_documents)


In [ ]:
print(train_bow_embeddings.shape)

(186, 3000)


In [ ]:
vocab = vectorizer.get_feature_names()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
train_contextualized_embeddings = bert_embeddings_from_list(preprocessed_documents, \
                                                            "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")


Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:01<?, ?it/s]

In [ ]:
qt = TopicModelDataPreparation()

training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)

### Combined TM 학습하기

In [ ]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=20, num_epochs=20)
ctm.fit(training_dataset)

Epoch: [20/20]	 Seen Samples: [3720/3720]	Train Loss: 2425.5570396505377	Time: 0:00:00.572145: : 20it [00:12,  1.58it/s]


In [ ]:
ctm.get_topics(7)

defaultdict(list,
            {0: ['900', '사람', '이건', '이미지', '조니워커', '종이', '450'],
             1: ['하면', '개월', '많이', '스푼', '모유', '요즘', '있어요'],
             2: ['섭취', '단백질', '운동', '기능', '도움', '하기', '좋은'],
             3: ['카페', '라이스', '전통', '입장', 'ㅠㅠ', '화이', '칭찬'],
             4: ['사람', '있는데요', '두유', '함량', '이러한', '오트밀', '좋겠더라고요'],
             5: ['유청', '진짜', '하이', '에너지', '주식', '체력', '좋아해서'],
             6: ['두유', '보니', '있는데', '에스프레소', '그냥', '오틀리', '에도'],
             7: ['자기방어', '구매', '에프', '앱솔', '저녁', '생겨서', '에겐'],
             8: ['900', 'mg', '되어', '이나', '체중', '경우', '있다'],
             9: ['인증', '이나', '렉스', '당뇨', '가수분해', '하면서', '관리'],
             10: ['바리스타', '귀리', '음료', '이나', '모유', '해서', '요거트'],
             11: ['우유', '어메이징', '당류', '식물성', '같아요', '요즘', '인증'],
             12: ['유광', '오전', '없으면', '이렇게', '국가수', '분위기', '생겨서'],
             13: ['만나', '최대', '030원', '에서만', '메인', '파티', '예쁘게'],
             14: ['개월', '우리', '앱솔', '최초', '분유', '340', '상추'],
             15: ['ml', '제품',

In [ ]:
# ctm.get_topics(10)

In [ ]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
Sampling: [10/10]: : 10it [00:03,  2.82it/s]
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
